# Initial setup

Import modules

In [57]:
import csv
import sys
import datetime

Set variables for this run

In [58]:
namespaceUri = 'http://rs.tdwg.org/ac/terms/'
database = 'audubon'
vocabulary = 'ac'
namespace = 'terms'
date_issued = '2019-09-30'
local_offset_from_utc = '-05:00'
versions = database + '-versions'
modifications_filename = 'mods.csv'
version_namespace = namespaceUri + 'version/'

Define utility functions

In [59]:
def readCsv(filename):
    fileObject = open(filename, 'r', newline='', encoding='utf-8')
    readerObject = csv.reader(fileObject)
    array = []
    for row in readerObject:
        array.append(row)
    fileObject.close()
    return array

def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()

    # returns a list with first item Boolean and second item the index
def findColumnWithHeader(header_row_list, header_label):
    found = False
    for column_number in range(0, len(header_row_list)):
        if header_row_list[column_number] == header_label:
            found = True
            found_column = column_number
    if found:
        return [True, found_column]
    else:
        return [False, 0]
    
def isoTime(offset):
    currentTime = datetime.datetime.now()
    return currentTime.strftime("%Y-%m-%dT%H:%M:%S") + offset

# Extract information from metadata files

**Note for all tables:** row 0 (the first row) is a header row.

The table containing the **modifications** (term additions and changes) looks like this:

![](mods-table.png)

The `term_localName` column is the primary key for this table.

The table containing **current terms metadata** looks like this:

![](current-terms-table1.png)
![](current-terms-table2.png)

Notice that all of the column headers from `label` onwards to the right are the same. The column headers to the left of `label` are ideosyncratic for the table type and must be handled specially.  The `term_localName` column is the primary key for this table.

The table containing **versions metadata** looks like this:

![](versions-table1.png)
![](versions-table2.png)

As with the current terms table, all of the column headers from `label` onwards to the right are the same as the previous two tables. The column headers to the left of `label` are ideosyncratic for the table type and must be handled specially. The `versionLocalName` column is the primary key for this table.  The `term_localName` column is a foreign key that relates rows in this table to rows in the other two tables.  

## Read in the tables of current terms and of modifications

In [60]:
term_metadata_filename = database + '/' + database + '.csv'
term_metadata = readCsv(term_metadata_filename)

modifications_metadata = readCsv(modifications_filename)
print('Headers: ', modifications_metadata[0])

Headers:  ['term_localName', 'label', 'rdfs_comment', 'skos_scopeNote', 'dcterms_description', 'rdf_type', 'tdwgutility_organizedInClass', 'tdwgutility_decision', 'tdwgutility_layer', 'tdwgutility_required', 'tdwgutility_repeatable']


The script loads data only on the basis of the column names and not their position.  So a number of variables are defined that hold the column numbers for various fields.  

Find which column numbers in the modifications file and the metadata file hold the term local name.  This column is the primary key for the table and therefore is used to match rows for terms that need to be modified with the corresponding rows in the current term metadata table.  The `term_localName` column is the only one in the modifications table that is not potentially a vocabulary-specific metadata field.

In [61]:
result = findColumnWithHeader(modifications_metadata[0], 'term_localName')
if result[0] == False:
    print('The modifications file does not have a term_localName column')
    sys.exit
else:
    mods_local_name = result[1]

# don't error trap here because all existing files should have a local name column header
result = findColumnWithHeader(term_metadata[0], 'term_localName')
metadata_localname_column = result[1]

Create a list of the local names of terms to be added or modified

In [62]:
mods_term_localName = []
for term_number in range(1, len(modifications_metadata)):
    mods_term_localName.append(modifications_metadata[term_number][mods_local_name])
print(mods_term_localName)

['attributionLogoURL', 'captionA', 'captureDevice', 'commenter', 'commenterLiteral']


Find out which terms are new terms and which are modified old terms.  Create a list for each.

In [63]:
new_terms = []
modified_terms = []
for test_term in mods_term_localName:
    found = False
    for term in term_metadata:
        if test_term == term[metadata_localname_column]:
            found = True
            modified_terms.append(test_term)
    if not found:
        new_terms.append(test_term)
print('New terms: ', new_terms)
print('Modified terms: ', modified_terms)

New terms:  ['captionA']
Modified terms:  ['attributionLogoURL', 'captureDevice', 'commenter', 'commenterLiteral']


# Load versions metadata and determine positions of special columns

Read in the term versions metadata file

In [64]:
term_versions_metadata_filename = versions + '/' + versions + '.csv'
term_versions_metadata = readCsv(term_versions_metadata_filename)
print(term_versions_metadata[0])

['document_modified', 'version', 'versionLocalName', 'version_isDefinedBy', 'version_issued', 'version_status', 'replaces_version', 'replaces1_version', 'replaces2_version', 'label', 'rdfs_comment', 'skos_scopeNote', 'dcterms_description', 'rdf_type', 'tdwgutility_organizedInClass', 'tdwgutility_decision', 'tdwgutility_layer', 'tdwgutility_required', 'tdwgutility_repeatable', 'term_localName']


Find the positions of the ideosyncratic version columns.  Recall:

![](versions-table1.png)
![](versions-table2.png)

In [65]:
version_modified = findColumnWithHeader(term_versions_metadata[0], 'document_modified')[1]
version_column = findColumnWithHeader(term_versions_metadata[0], 'version')[1]
version_local_name = findColumnWithHeader(term_versions_metadata[0], 'versionLocalName')[1]
version_isDefinedBy = findColumnWithHeader(term_versions_metadata[0], 'version_isDefinedBy')[1]
version_issued = findColumnWithHeader(term_versions_metadata[0], 'version_issued')[1]
version_status = findColumnWithHeader(term_versions_metadata[0], 'version_status')[1]
version_term_local_name_column = findColumnWithHeader(term_versions_metadata[0], 'term_localName')[1]


# Update the master versions table

This is the primary metadata table for generating information about versions

## Supersede old versions of the modified terms

Go through each version and supersede any that match the local names of the modified terms

In [66]:
for term in modified_terms:
    for version_row in range(1, len(term_versions_metadata)):
        if term_versions_metadata[version_row][version_term_local_name_column] == term:
            print(version_row, term)
            term_versions_metadata[version_row][version_status] = 'superseded'
            # the 0th column always contains the time modified value
            term_versions_metadata[version_row][version_modified] = isoTime(local_offset_from_utc)

5 attributionLogoURL
7 captureDevice
8 commenter
9 commenterLiteral


## Create new versions of new and modified terms

Make sure that all columns in modified terms file are in the term versions file

In [67]:
for column in modifications_metadata[0]:
    result = findColumnWithHeader(term_versions_metadata[0], column)
    if result[0] == False:
        print('The modifications file is missing the ', column, ' column.')
        sys.exit

Create new versions list

In [68]:
newVersions = []

Create a row in the new term versions list for the added or modified terms

In [69]:
for row_number in range(1, len(modifications_metadata)):
    newVersion = []
    # create a column for every column in the term version file
    for column in term_versions_metadata[0]:
        # find the column in the modifications file that matches the version column and add its value
        result = findColumnWithHeader(modifications_metadata[0], column)
        if result[0] == True:
            newVersion.append(modifications_metadata[row_number][result[1]])
        else:
            newVersion.append('')
    # set the modification dateTime for the newly created version
    newVersion[version_modified] = isoTime(local_offset_from_utc)
    newVersions.append(newVersion) 

Insert metadata specific to the new versions

In [70]:
for rowNumber in range(0, len(newVersions)):
    newVersions[rowNumber][version_issued] = date_issued
    newVersions[rowNumber][version_status] = 'recommended'
    # need to add one to the row of modifications_metadata because it includes the header row
    newVersions[rowNumber][version_local_name] = modifications_metadata[rowNumber + 1][mods_local_name] + '-' + date_issued
    newVersions[rowNumber][version_isDefinedBy] = version_namespace
    newVersions[rowNumber][version_column] = version_namespace + modifications_metadata[rowNumber + 1][mods_local_name] + '-' + date_issued


Append the new versions to the old version file

In [71]:
revised_term_versions_metadata = term_versions_metadata + newVersions
writeCsv('temp_versions.csv', revised_term_versions_metadata)

# Load current terms metadata and determine positions of special columns

Read in the current term file

In [72]:
terms_metadata_filename = database + '/' + database + '.csv'
terms_metadata = readCsv(terms_metadata_filename)
print(term_metadata[0])

['document_modified', 'term_localName', 'term_isDefinedBy', 'term_created', 'term_modified', 'term_deprecated', 'replaces_term', 'replaces1_term', 'replaces2_term', 'label', 'rdfs_comment', 'skos_scopeNote', 'dcterms_description', 'rdf_type', 'tdwgutility_organizedInClass', 'tdwgutility_decision', 'tdwgutility_layer', 'tdwgutility_required', 'tdwgutility_repeatable']


Find the positions of the ideosyncratic version columns. Recall:

![](current-terms-table1.png)
![](current-terms-table2.png)

In [73]:
term_modified_dateTime = findColumnWithHeader(terms_metadata[0], 'document_modified')[1]
term_localName = findColumnWithHeader(terms_metadata[0], 'term_localName')[1]
term_modified = findColumnWithHeader(terms_metadata[0], 'term_modified')[1]
term_created = findColumnWithHeader(terms_metadata[0], 'term_created')[1]
term_isDefinedBy = findColumnWithHeader(terms_metadata[0], 'term_isDefinedBy')[1]

# Modify current terms metadata table

Each item in the term modifications list will either modify existing term metadata or add new term metadata

In [74]:
# step through each row in the modification metadata table and modify existing current terms when applicable
for mods_rownumber in range(1, len(modifications_metadata)):
    mods_localname_string = modifications_metadata[mods_rownumber][mods_local_name]
    modified = False
    for term_name in modified_terms:
        # only make a modification if it's on the list of terms to be modified
        if mods_localname_string == term_name:
            modified = True
    # this section of code modifies existing terms
    if modified:
        # find the row in the terms metadata file for the term to be modified
        for term_rownumber in range(1, len(terms_metadata)):
            if mods_localname_string == terms_metadata[term_rownumber][term_localName]:
                terms_metadata[term_rownumber][term_modified_dateTime] = isoTime(local_offset_from_utc)
                terms_metadata[term_rownumber][term_modified] = date_issued
                # replace every column that's in the modifications metadata
                for column_number in range(0, len(modifications_metadata[0])):
                    # find the column in the current terms metadata table that matches the modifications column and replace the current term's value
                    result = findColumnWithHeader(terms_metadata[0], modifications_metadata[0][column_number])
                    if result[0] == True:
                        terms_metadata[term_rownumber][result[1]] = modifications_metadata[mods_rownumber][column_number]
                    else:
                        pass # this shouldn't really happen since there already was a check that all columns existed in the versions table
                print(terms_metadata[term_rownumber])
    # this section of code adds new term metadata
    else: 
        newTermRow = []
        for column in range(0, len(terms_metadata[0])):
            newTermRow.append('')
        newTermRow[term_modified_dateTime] = isoTime(local_offset_from_utc)
        newTermRow[term_modified] = date_issued
        newTermRow[term_created] = date_issued
        newTermRow[term_isDefinedBy] = namespaceUri
        # replace every column that's in the modifications metadata
        for column_number in range(0, len(modifications_metadata[0])):
            # find the column in the current terms metadata table that matches the modifications column and replace the current term's value
            result = findColumnWithHeader(terms_metadata[0], modifications_metadata[0][column_number])
            if result[0] == True:
                newTermRow[result[1]] = modifications_metadata[mods_rownumber][column_number]
            else:
                pass # this shouldn't really happen since there already was a check that all columns existed in the versions table
        print(newTermRow)
        terms_metadata.append(newTermRow)
writeCsv('temp_terms.csv', terms_metadata)

['2019-10-17T15:54:09-05:00', 'attributionLogoURL', 'http://rs.tdwg.org/ac/terms/', '2013-10-28', '2019-09-30', '', '', '', '', 'Attribution URL', 'Goofy new definition of Att URL', '', 'Entering this URL into a browser should only result in the icon (not in a webpage including the icon).', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property', 'http://rs.tdwg.org/dwc/terms/attributes/Attribution', '', '1', 'No', 'No']
['2019-10-17T15:54:09-05:00', 'captionA', 'http://rs.tdwg.org/ac/terms/', '2019-09-30', '2019-09-30', '', '', '', '', 'Caption', 'As alternative or in addition to description, a caption is free-form text to be displayed together with (rather than instead of) a resource that is suitable for captions (especially images).', '', 'If both description and caption are present in the metadata, a description is typically displayed instead of the resource, a caption together with the resource. Often only one of description or caption is present; choose the term most appropriate fo